In [14]:
# Computational Methods
# Term Project: Kalman Filters for Energy Systems
# Stefan Deutsch, Viv Pitter, Ansel Klein
# Prof. Anthoff
# Due 5/12/17

# Parameters

R1A = 5 # Thermal resistance between Room 1 and ambient, deg C/kW
R12 = 3 # Thermal resistance between Room 1 and Room 2, deg C/kW
R23 = 3 # Thermal resistance between Room 2 and Room 3, deg C/kW
R3A = 5 # Thermal resistance between Room 3 and ambient, deg C/kW
C1 = 24  # Thermal capacitance of Room 1, kWh/deg C
C2 = 15  # Thermal capacitance of Room 2, kWh/deg C
C3 = 48  # Thermal capacitance of Room 3, kWh/deg C
P = 8  # Thermal power of heater, kW

# State space matrices
A = [-1/C1*(1/R1A + 1/R12) 1/(C1*R12) 0; 1/(C2*R12) -1/C2*(1/R12 + 1/R23) 1/(C2*R23); 0 1/(C3*R23) -1/C3*(1/R23 + 1/R3A)]
B = [1/(C1*R1A) P/(2*C1); 0 0; 1/(C3*R3A) P/(2*C3)]
C = [0 1 0]
D = [0 0]

# Compute observability Matrix for 3-state system and rank
O = [C; C*A; C*A^2]
r = rank(O)  # could also do this by inspection?
println("Rank of the observability matrix is $r.")

# Upload data here

# t = time series vector
t = collect(1:1:10)'
# T_A = vector of ambient temperature data
T_A = [11 11 11 12 12 12 12 13 12 11]
# s = 0 or 1 depending on whether heater is on or off
s = [1 1 1 1 0 0 0 0 0 1 ]
# T_2 = vector of sensed temperature data in room 2
T_2 = [20 21 23 24 23 22 21 20 20 21]

# N = covariance of sensor noise

u = [T_A; s]  # collect ambient temperature and heater state into input matrix
ym = T_2  # temperature of room 2 is state of interest and thus the measured output

# x0 = mean initial states (arbitrary)
# sig0 = covariance of initial states (arbitrary)

# Kalman Filter

# Noise covariances
W = diagm([.05; .02; .05])  # Covariance of model inaccuracy / process noise  We tune this to our liking.
N = 1.5  # just an example right now
sig0 = 1.5*eye(3) # just an example right now

# Estimated Initial Conditions
# x_hat0 = x0
x_hat0 = [20; 20; 20]  # just an example right now

# Simulate Kalman Filter
# original equation was [tsim,states] = ode45(@(t,x) ode_kf(t,x,A,B,C,D,input_data,W,N),t,states0) where 'states' combined x and sigma
# i.e. [tsim, xsim, sigsim] = our_ode(kalman(x_hat,sig,A,B,C,D,ym,u,W,N), t, x_hat0, sig0)
(x_hat, sig) = Euler(kalman,A,B,C,D,ym,u,W,N,t,x_hat0,sig0)  # using Euler's method right now

# Parse States
T1_hat = x_hat[1,:]'
T2_hat = x_hat[2,:]'
T3_hat = x_hat[3,:]'
# Compute the upper and lower bounds on T1 and T3
bound1 = zeros(length(t))'
bound3 = zeros(length(t))'
for k in 1:length(t)
    bound1[k] = sig[k][1,1]
    bound3[k] = sig[k][3,3]
end

# Plot results
println(t)
println(s)
println(T_A)
println(T1_hat)
println(T2_hat)
println(T3_hat)
# Plot true and estimated T2 versus time
println(T_2)
println(T2_hat)
# Plot estimated T1 and T3 plus/minus 1 sigma
println(T1_hat+bound1)
println(T1_hat-bound1)
println(T3_hat+bound3)
println(T3_hat-bound3)
# Plot error between true and estimated T2
println(T2_hat-T_2)

# Calculate RMSE between true and estimated T2


# Extended Kalman Filter if we so choose


Rank of the observability matrix is 3.
[1 2 3 4 5 6 7 8 9 10]
[1 1 1 1 0 0 0 0 0 1]
[11 11 11 12 12 12 12 13 12 11]
[20.0 20.0917 20.2174 20.4795 20.8954 21.1292 21.2996 21.362 21.2963 21.215]
[20.0 20.0 19.9275 19.7591 19.625 19.6216 19.7008 19.8077 19.8853 19.9558]
[20.0 20.0458 20.1203 20.3231 20.6822 20.9608 21.1864 21.3069 21.2891 21.2578]
[20 21 23 24 23 22 21 20 20 21]
[20.0 20.0 19.9275 19.7591 19.625 19.6216 19.7008 19.8077 19.8853 19.9558]
[21.5 21.575 21.6844 21.9287 22.3243 22.5347 22.6784 22.7114 22.6147 22.5022]
[18.5 18.6083 18.7505 19.0303 19.4664 19.7237 19.9208 20.0127 19.978 19.9279]
[21.5 21.5625 21.6526 21.8685 22.2365 22.5189 22.7425 22.8558 22.8264 22.7806]
[18.5 18.5292 18.588 18.7778 19.128 19.4027 19.6302 19.758 19.7519 19.735]
[0.0 -1.0 -3.0725 -4.24094 -3.37504 -2.37842 -1.29917 -0.192313 -0.114713 -1.04423]


In [13]:
function kalman(x_hat,sig,A,B,C,D,ym,u,W,N)  # kalman filter for a general system

    # Compute Kalman Gain
    L = sig*C'*inv(N)

    # Kalman Filter equations
    y_hat = C*x_hat + D*u
    x_hat_dot =  A*x_hat + B*u + L*(ym - y_hat)

    # Riccati Equation for Sigma
    sig_dot = sig*A' + A*sig + W - sig*C'*inv(N)*C*sig

    # Return values
    return x_hat_dot, sig_dot
end

kalman (generic function with 1 method)

In [12]:
function Euler(kf,A,B,C,D,ym,u,W,N,t,x_hat0,sig0)  # Euler's method ODE solver for our system
    h = 1  # step size
    x_hat = zeros(3,length(t))  # vector of corresponding state values
    sig = fill(zeros(3,3), 1, length(t))  # sigma is a vector of 3 by 3 matrices
    x_hat[:,1] = x_hat0  # initial values
    sig[1] = sig0
    for i in 2:length(t)  # euler's method
        (x_hat_dot, sig_dot) = kf(x_hat[:,i-1],sig[i-1],A,B,C,D,ym[i-1],u[:,i-1],W,N)  # dx/dt from kalman filter eqn
        x_hat[:,i] = x_hat[:,i-1] + h*x_hat_dot  # increment states and sigma values
        sig[i] = sig[i-1] + h*sig_dot
    end
    return x_hat, sig
end

Euler (generic function with 1 method)